In [1]:
# Copyright 2021 NVIDIA Corporation. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

In [2]:
# External dependencies
import os

import cudf  # cuDF is an implementation of Pandas-like Dataframe on GPU
# import rmm

import numpy as np
import nvtabular as nvt

from sklearn.model_selection import train_test_split

In [3]:
INPUT_DATA_DIR = os.environ.get(
    "INPUT_DATA_DIR", os.path.expanduser("~/nvt-examples/end-to-end-poc/data/")
)

## Read Movie and Rating Features

In [4]:
movie_features = cudf.read_parquet(os.path.join(INPUT_DATA_DIR, "movie_features.parquet"))
movie_features.head()

,movieId,tags_unique,genres,tags_nunique
0,1,"[477, 581, 640, 1857, 2175, 2817, 3538, 4395, ...","[3, 4, 5, 6, 10]",126
1,2,"[1206, 4448, 5069, 5213, 7883, 8912, 9116, 925...","[3, 5, 10]",44
2,3,"[2196, 4356, 4448, 6484, 11460, 12274, 17647, ...","[6, 16]",23
3,4,"[4448, 31525, 34749, 34981, 39134, 48169, 6086...","[6, 9, 16]",8
4,5,"[4448, 5693, 6977, 8912, 23756, 25354, 28604, ...",[6],20


In [5]:
ratings_features = cudf.read_parquet(os.path.join(INPUT_DATA_DIR, "ratings_features.parquet"))
ratings_features.head()

,day,interaction,userId,movieId,rating,timestamp
0,4146,True,1,296,5.0,1147880044
1,4146,True,1,306,3.5,1147868817
2,4146,True,1,307,5.0,1147868828
3,4146,True,1,665,5.0,1147878820
4,4146,True,1,899,3.5,1147868510


## Join Ratings With Features

In [6]:
joined_features = movie_features.merge(ratings_features)
joined_features.head()

,movieId,tags_unique,genres,tags_nunique,day,interaction,userId,rating,timestamp
0,27156,"[7739, 18057]","[3, 5, 6, 7]",2,8378,True,235,5.0,1513481086
1,31658,"[1135, 2649, 5791, 8723, 22401, 23840, 23943]",[15],7,8358,True,235,5.0,1511746293
2,33679,[57264],[9],1,8372,True,235,5.0,1512965368
3,40815,"[25864, 36224, 39517, 52690, 61371, 67439]","[6, 9]",6,8371,True,235,5.0,1512890306
4,7320,"[4050, 4567, 6022, 6224, 8396, 12169, 12590, 1...","[9, 15, 16]",28,6896,True,91,3.5,1385507997


In [7]:
sampled_indices = np.array([np.random.randint(0,nunique) for nunique in joined_features["tags_nunique"].to_pandas()], dtype=np.int32)

In [8]:
sampled_tags = np.array([tags[index] for index, tags in zip(sampled_indices, joined_features["tags_unique"].to_pandas())])

In [9]:
joined_features["sampled_tag"] = sampled_tags

In [10]:
joined_features = joined_features.drop(labels=["tags_unique", "tags_nunique", "interaction", "rating"], axis=1)

In [11]:
joined_features = joined_features.sort_values("timestamp")

In [12]:
joined_features.head()

,movieId,genres,day,userId,timestamp,sampled_tag
156270,47,"[15, 18]",0,2262,789652009,38971
999957,57,"[9, 15]",385,13424,822873600,21387
999979,11,"[6, 9, 16]",385,13424,822873600,17647
3287389,32,"[15, 17, 18]",385,42937,822873600,4171
7850959,18,[6],385,102689,822873600,591


In [13]:
grouped_examples = joined_features.groupby(["userId", "day"]).agg(
    {
        "sampled_tag": "collect",
        "genres": "collect",
        "timestamp": "collect",
        "movieId": ["collect", "count"]
    }
)

In [14]:
grouped_examples.head()

sampled_tag  \
                                                       collect   
userId day                                                       
1      4146  [43870, 39284, 9623, 28742, 42431, 59725, 1882...   
2      4071  [43870, 18452, 43415, 45148, 497, 57559, 69008...   
3      7521  [44855, 60851, 36606, 47056, 57629, 64877, 219...   
       7688  [34113, 3545, 46964, 50807, 36224, 22485, 4678...   
       8045  [3219, 13153, 47665, 2551, 4459, 40409, 51083,...   

                                                        genres  \
                                                       collect   
userId day                                                       
1      4146  [[6], [6], [3, 6, 17], [9, 16], [3, 9], [2, 3,...   
2      4071  [[6], [6], [6], [6], [6, 9], [9], [7, 9, 15], ...   
3      7521  [[6, 9, 16], [9], [2, 6, 7, 9, 17], [3, 4, 5, ...   
       7688  [[12], [12, 18], [7, 9, 18], [7, 9], [6, 9], [...   
       8045  [[7, 9, 17, 18], [7, 9, 18], [7, 9], [9], [7, ...   

                                                     timestamp  \
                                                       collect   
userId day                                                       
1      4146  [1147868053, 1147868097, 1147868414, 114786846...   
2      4071  [1141415528, 1141415566, 1141415576, 114141558...   
3      7521  [1439472199, 1439472203, 1439472211, 143947221...   
       7688  [1453904021, 1453904031, 1453904046, 145390404...   
       8045  [1484753654, 1484753766, 1484753808, 148475384...   

                                                       movieId        
                                                       collect count  
userId day                                                            
1      4146  [5952, 1653, 1250, 6539, 6377, 3448, 1088, 899...    53  
2      4071  [5952, 497, 1374, 1653, 2640, 5445, 151, 236, ...   125  
3      7521  [356, 593, 1270, 1, 2571, 260, 318, 1196, 527,...   179  
       7688  [1206, 1208, 44191, 32587, 40815, 36529, 45186...    10  
       8045  [1089, 4011, 741, 778, 111, 214, 293, 1252, 33...    22

In [15]:
# TODO: Rename the columns

In [16]:
grouped_examples.columns

MultiIndex([('sampled_tag', 'collect'),
            (     'genres', 'collect'),
            (  'timestamp', 'collect'),
            (    'movieId', 'collect'),
            (    'movieId',   'count')],
           )

In [21]:
grouped_examples.head()

sampled_tag  \
                                                       collect   
userId day                                                       
1      4146  [43870, 39284, 9623, 28742, 42431, 59725, 1882...   
2      4071  [43870, 18452, 43415, 45148, 497, 57559, 69008...   
3      7521  [44855, 60851, 36606, 47056, 57629, 64877, 219...   
       7688  [34113, 3545, 46964, 50807, 36224, 22485, 4678...   
       8045  [3219, 13153, 47665, 2551, 4459, 40409, 51083,...   

                                                        genres  \
                                                       collect   
userId day                                                       
1      4146  [[6], [6], [3, 6, 17], [9, 16], [3, 9], [2, 3,...   
2      4071  [[6], [6], [6], [6], [6, 9], [9], [7, 9, 15], ...   
3      7521  [[6, 9, 16], [9], [2, 6, 7, 9, 17], [3, 4, 5, ...   
       7688  [[12], [12, 18], [7, 9, 18], [7, 9], [6, 9], [...   
       8045  [[7, 9, 17, 18], [7, 9, 18], [7, 9], [9], [7, ...   

                                                     timestamp  \
                                                       collect   
userId day                                                       
1      4146  [1147868053, 1147868097, 1147868414, 114786846...   
2      4071  [1141415528, 1141415566, 1141415576, 114141558...   
3      7521  [1439472199, 1439472203, 1439472211, 143947221...   
       7688  [1453904021, 1453904031, 1453904046, 145390404...   
       8045  [1484753654, 1484753766, 1484753808, 148475384...   

                                                       movieId        
                                                       collect count  
userId day                                                            
1      4146  [5952, 1653, 1250, 6539, 6377, 3448, 1088, 899...    53  
2      4071  [5952, 497, 1374, 1653, 2640, 5445, 151, 236, ...   125  
3      7521  [356, 593, 1270, 1, 2571, 260, 318, 1196, 527,...   179  
       7688  [1206, 1208, 44191, 32587, 40815, 36529, 45186...    10  
       8045  [1089, 4011, 741, 778, 111, 214, 293, 1252, 33...    22

In [22]:
len(grouped_examples[grouped_examples[("movieId", "count")] > 1])

498253

In [23]:
# TODO: Rename columns from tuples to strings

In [24]:
grouped_examples["target_item"] = grouped_examples[("movieId", "collect")].list.get(-1)

In [25]:
grouped_examples[("sampled_tag", "collect")] = \
    np.array([values[:-1] for values in grouped_examples[("sampled_tag", "collect")].to_pandas()], dtype=object)

In [27]:
grouped_examples[("genres", "collect")] = \
    np.array([values[:-1] for values in grouped_examples[("genres", "collect")].to_pandas()], dtype=object)

In [28]:
grouped_examples[("timestamp", "collect")] = \
    np.array([values[:-1] for values in grouped_examples[("timestamp", "collect")].to_pandas()], dtype=object)

In [29]:
grouped_examples[("movieId", "collect")] = \
    np.array([values[:-1] for values in grouped_examples[("movieId", "collect")].to_pandas()], dtype=object)

In [35]:
grouped_examples[("movieId", "count")] = grouped_examples[("movieId", "count")] - 1

In [33]:
def flatten_genres(g):
    if len(g) > 0:
        return np.concatenate(g).ravel()
    else:
        return []

grouped_examples[("genres", "collect")] = \
    grouped_examples[("genres", "collect")].to_pandas().map(flatten_genres)

In [36]:
grouped_examples

sampled_tag  \
                                                       collect   
userId day                                                       
1      4146  [43870, 39284, 9623, 28742, 42431, 59725, 1882...   
2      4071  [43870, 18452, 43415, 45148, 497, 57559, 69008...   
3      7521  [44855, 60851, 36606, 47056, 57629, 64877, 219...   
       7688  [34113, 3545, 46964, 50807, 36224, 22485, 4678...   
       8045  [3219, 13153, 47665, 2551, 4459, 40409, 51083,...   
...                                                        ...   
162538 7513  [24119, 2816, 6658, 58706, 41345, 46784, 62147...   
162539 2378  [4171, 38530, 43221, 62981, 62981, 55568, 3025...   
162540 5315  [41241, 48960, 2816, 64058, 8898, 30253, 46862...   
       5317                        [61752, 1191, 47665, 24623]   
162541 5223  [18782, 59536, 49414, 50070, 53264, 63325, 237...   

                                                        genres  \
                                                       collect   
userId day                                                       
1      4146  [6, 6, 3, 6, 17, 9, 16, 3, 9, 2, 3, 6, 10, 3, ...   
2      4071  [6, 6, 6, 6, 6, 9, 9, 7, 9, 15, 7, 9, 2, 6, 6,...   
3      7521  [6, 9, 16, 9, 2, 6, 7, 9, 17, 3, 4, 5, 6, 10, ...   
       7688  [12, 12, 18, 7, 9, 18, 7, 9, 6, 9, 1, 1, 12, 2...   
       8045  [7, 9, 17, 18, 7, 9, 18, 7, 9, 9, 7, 9, 9, 18,...   
...                                                        ...   
162538 7513  [6, 9, 16, 9, 2, 6, 7, 9, 17, 9, 14, 6, 16, 2,...   
162539 2378  [9, 19, 9, 12, 12, 6, 9, 6, 9, 12, 6, 16, 6, 1...   
162540 5315  [9, 16, 17, 2, 6, 8, 6, 16, 9, 17, 12, 15, 17,...   
       5317                         [3, 9, 6, 9, 16, 6, 2, 18]   
162541 5223  [9, 16, 19, 6, 9, 9, 6, 7, 9, 18, 8, 2, 18, 2,...   

                                                     timestamp  \
                                                       collect   
userId day                                                       
1      4146  [1147868053, 1147868097, 1147868414, 114786846...   
2      4071  [1141415528, 1141415566, 1141415576, 114141558...   
3      7521  [1439472199, 1439472203, 1439472211, 143947221...   
       7688  [1453904021, 1453904031, 1453904046, 145390404...   
       8045  [1484753654, 1484753766, 1484753808, 148475384...   
...                                                        ...   
162538 7513  [1438780751, 1438780754, 1438780759, 143878083...   
162539 2378  [995149720, 995149760, 995149788, 995149788, 9...   
162540 5315  [1248854959, 1248855584, 1248856442, 124885702...   
       5317   [1249028584, 1249028593, 1249029673, 1249029678]   
162541 5223  [1240946758, 1240946774, 1240947363, 124094737...   

                                                       movieId        \
                                                       collect count   
userId day                                                             
1      4146  [5952, 1653, 1250, 6539, 6377, 3448, 1088, 899...    52   
2      4071  [5952, 497, 1374, 1653, 2640, 5445, 151, 236, ...   124   
3      7521  [356, 593, 1270, 1, 2571, 260, 318, 1196, 527,...   178   
       7688  [1206, 1208, 44191, 32587, 40815, 36529, 45186...     9   
       8045  [1089, 4011, 741, 778, 111, 214, 293, 1252, 33...    21   
...                                                        ...   ...   
162538 7513  [356, 593, 1270, 527, 2324, 2858, 2762, 4993, ...    67   
162539 2378  [1721, 480, 1784, 1198, 1282, 608, 3363, 2022,...    42   
162540 5315  [3005, 5400, 4167, 4372, 43936, 32, 6383, 8644...    28   
       5317                           [6377, 8961, 4720, 8957]     4   
162541 5223  [1729, 261, 969, 2105, 2161, 3255, 914, 1249, ...   102   

            target_item  
                         
userId day               
1      4146        7361  
2      4071        2150  
3      7521       37729  
       7688        4344  
       8045       27773  
...                 ...  
162538 7513        6668  
162539 